# A geocomputational to monitor regional development in Bolivia

## Setup

In [2]:
# Load libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.ticker import FormatStrFormatter
from matplotlib_scalebar.scalebar import ScaleBar
from pylab import rcParams
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 72

import seaborn as sns
#sns.set_style("darkgrid")
#sns.set_context(context="paper", font_scale=1.5, rc=None)
#sns.set(font="serif")

import plotly.express as px
import plotly.graph_objects as go

import geopandas as gpd


import libpysal
from libpysal  import weights
from libpysal.weights import Queen

import esda
from esda.moran import Moran, Moran_Local

import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster, plot_local_autocorrelation
from splot.libpysal import plot_spatial_weights

from giddy.directional import Rose

import statsmodels.api as sm
import statsmodels.formula.api as smf

from spreg import OLS
from spreg import MoranRes
from spreg import ML_Lag
from spreg import ML_Error 

from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap

import warnings
warnings.filterwarnings('ignore') 

## Import data

In [3]:
df = pd.read_stata("/work/project2021o/data/rawData/BaseDatosAtlasMunicipalODSBolivia2020_Stata15.dta")
df

,id,municipio,dep,imds,rank_imds,poblacion_2020,urbano_2012,sdg1_1_ee,sdg1_1_nbi,sdg1_2_pm,sdg1_4_ssb,sdg2_2_dm,sdg2_2_smu,sdg2_4_su,sdg2_4_t,sdg3_1_p,sdg3_2_mi,sdg3_2_mn,sdg3_3_c,sdg3_3_d,sdg3_3_m,sdg3_3_t,sdg3_3_vih,sdg3_7_fa,sdg4_1_ash,sdg4_1_asm,sdg4_4_es,sdg4_6_alfab,sdg4_c_pci,sdg4_c_pcs,sdg5_1_paes,sdg5_1_pej,sdg5_1_ppm,sdg5_5_pp,sdg6_1_ca,sdg6_2_cs,sdg6_3_tar,sdg7_1_cee,sdg7_1_cep,sdg7_1_elc,sdg7_3_ecep,sdg8_4_mcc,sdg8_5_ph,sdg8_5_pm,sdg8_6_hnn,sdg8_6_mnn,sdg8_10_b,sdg8_11_mm,sdg9_1_rutas,sdg9_5_c,sdg9_5_pt,sdg9_c_crdm,sdg9_c_rb,sdg10_2_egini,sdg10_2_ge,sdg10_2_nhe,sdg11_1_h,sdg11_1_sss,sdg11_2_atc,sdg13_1_vcc,sdg13_2_ectp,sdg13_2_td,sdg15_1_ap,sdg15_5_pb,sdg16_1_vh,sdg16_6_ce,sdg16_9_nrc,sdg17_1_il,sdg17_5_ip,sdg1_1_ee_abs,sdg1_1_nbi_abs,sdg1_2_pm_abs,sdg1_4_ssb_abs,sdg2_2_dm_abs,sdg2_2_smu_abs,sdg2_4_su_abs,sdg2_4_t_abs,sdg3_1_p_abs,sdg3_2_mi_abs,sdg3_2_mn_abs,sdg3_3_c_abs,sdg3_3_d_abs,sdg3_3_m_abs,sdg3_3_t_abs,sdg3_3_vih_abs,sdg3_7_fa_abs,sdg4_1_ash_abs,sdg4_1_asm_abs,sdg4_4_es_abs,sdg4_6_alfab_abs,sdg4_c_pci_abs,sdg4_c_pcs_abs,sdg6_1_ca_abs,sdg6_2_cs_abs,sdg7_1_cee_abs,sdg7_1_cep_abs,sdg7_1_elc_abs,sdg7_3_ecep_abs,sdg8_4_mcc_abs,sdg8_5_ph_abs,sdg8_5_pm_abs,sdg8_6_hnn_abs,sdg8_6_mnn_abs,sdg8_10_b_abs,sdg9_1_rutas_abs,sdg9_5_c_abs,sdg9_5_pt_abs,sdg9_c_crdm_abs,sdg9_c_rb_abs,sdg10_2_nhe_abs,sdg11_1_h_abs,sdg11_1_sss_abs,sdg11_2_atc_abs,sdg13_2_ectp_abs,sdg13_2_td_abs,sdg15_1_ap_abs,sdg15_5_pb_abs,sdg16_1_vh_abs,sdg16_6_ce_abs,sdg16_9_nrc_abs,sdg17_5_ip_abs,sdg1_1_ee_norm,sdg1_1_nbi_norm,sdg1_2_pm_norm,sdg1_4_ssb_norm,sdg2_2_dm_norm,sdg2_2_smu_norm,sdg2_4_su_norm,sdg2_4_t_norm,sdg3_1_p_norm,sdg3_2_mi_norm,sdg3_2_mn_norm,sdg3_3_c_norm,sdg3_3_d_norm,sdg3_3_m_norm,sdg3_3_t_norm,sdg3_3_vih_norm,sdg3_7_fa_norm,sdg4_1_ash_norm,sdg4_1_asm_norm,sdg4_4_es_norm,sdg4_6_alfab_norm,sdg4_c_pci_norm,sdg4_c_pcs_norm,sdg5_1_paes_norm,sdg5_1_pej_norm,sdg5_1_ppm_norm,sdg5_5_pp_norm,sdg6_1_ca_norm,sdg6_2_cs_norm,sdg6_3_tar_norm,sdg7_1_cee_norm,sdg7_1_cep_norm,sdg7_1_elc_norm,sdg7_3_ecep_norm,sdg8_4_mcc_norm,sdg8_5_ph_norm,sdg8_5_pm_norm,sdg8_6_hnn_norm,sdg8_6_mnn_norm,sdg8_10_b_norm,sdg8_11_mm_norm,sdg9_1_rutas_norm,sdg9_5_c_norm,sdg9_5_pt_norm,sdg9_c_crdm_norm,sdg9_c_rb_norm,sdg10_2_egini_norm,sdg10_2_ge_norm,sdg10_2_nhe_norm,sdg11_1_h_norm,sdg11_1_sss_norm,sdg11_2_atc_norm,sdg13_1_vcc_norm,sdg13_2_ectp_norm,sdg13_2_td_norm,sdg15_1_ap_norm,sdg15_5_pb_norm,sdg16_1_vh_norm,sdg16_6_ce_norm,sdg16_9_nrc_norm,sdg17_1_il_norm,sdg17_5_ip_norm,indice_ods1,indice_ods2,indice_ods3,indice_ods4,indice_ods5,indice_ods6,indice_ods7,indice_ods8,indice_ods9,indice_ods10,indice_ods11,indice_ods13,indice_ods15,indice_ods16,indice_ods17
0,10101,Sucre,Chuquisaca,69.599998,5,295476,91.400002,23.799999,26.900000,0.046,79.500000,14.800000,28.500000,NaN,NaN,83.000000,19.799999,24.700001,1.1,0.0,0.000000,57.299999,266.000000,44.299999,4.4,2.6,39.599998,94.800003,96.199997,77.599998,1.7,0.9,1.1,0.7,95.699997,93.900002,60.900002,93.599998,332,89.500000,1.3,4.900000,61.400002,43.900002,10.600000,16.400000,41.299999,9.6,3,4.1,21.200001,80.300003,7.5,0.32,110.0,7.000000,23.700001,17.900000,281.700012,0.7,1.300000,0.00,0.000000,0.000,2.4,86.199997,94.199997,15.9,671,17473.0,67621,6128,14316,4240,22626,52183.0,51.0,1407.0,109,136,940,0,6,163,297,701,764,446,61864,9466,11,314,12305,17250,17038,93191336.0,7299,353329,3107.0,38014,62200,3195,5300,119,3,3365,47.0,13729,2103,17148,16396,12393,80144,353329,0,0,0,20,737627253.0,1865,191011393.0,74.959999,72.209999,88.309998,79.500000,71.849998,19.840000,0.000000,0.00,79.860001,83.750000,100.000000,89.050003,100.0,99.980003,63.900002,36.169998,71.730003,54.709999,73.239998,100.000000,82.510002,96.180000,77.570000,100.0,86.309998,100.000000,55.590000,93.959999,93.570000,60.860001,91.419998,58.250000,91.730003,84.279999,88.150002,0.980000,29.799999,91.839996,73.599998,68.839996,72.290001,100.0,63.020000,26.459999,78.540001,64.739998,70.529999,54.209999,88.370003,69.620003,86.120003,33.380001,75.080002,94.769997,100.000000,0.00000

In [5]:
df.describe().round(2)

,id,imds,rank_imds,poblacion_2020,urbano_2012,sdg1_1_ee,sdg1_1_nbi,sdg1_2_pm,sdg1_4_ssb,sdg2_2_dm,sdg2_2_smu,sdg2_4_su,sdg2_4_t,sdg3_1_p,sdg3_2_mi,sdg3_2_mn,sdg3_3_c,sdg3_3_d,sdg3_3_m,sdg3_3_t,sdg3_3_vih,sdg3_7_fa,sdg4_1_ash,sdg4_1_asm,sdg4_4_es,sdg4_6_alfab,sdg4_c_pci,sdg4_c_pcs,sdg5_1_paes,sdg5_1_pej,sdg5_1_ppm,sdg5_5_pp,sdg6_1_ca,sdg6_2_cs,sdg6_3_tar,sdg7_1_cee,sdg7_1_cep,sdg7_1_elc,sdg7_3_ecep,sdg8_4_mcc,sdg8_5_ph,sdg8_5_pm,sdg8_6_hnn,sdg8_6_mnn,sdg8_10_b,sdg8_11_mm,sdg9_1_rutas,sdg9_5_c,sdg9_5_pt,sdg9_c_crdm,sdg9_c_rb,sdg10_2_egini,sdg10_2_ge,sdg10_2_nhe,sdg11_1_h,sdg11_1_sss,sdg11_2_atc,sdg13_1_vcc,sdg13_2_ectp,sdg13_2_td,sdg15_1_ap,sdg15_5_pb,sdg16_1_vh,sdg16_6_ce,sdg16_9_nrc,sdg17_1_il,sdg17_5_ip,sdg1_1_ee_abs,sdg1_1_nbi_abs,sdg1_2_pm_abs,sdg1_4_ssb_abs,sdg2_2_dm_abs,sdg2_2_smu_abs,sdg2_4_su_abs,sdg2_4_t_abs,sdg3_1_p_abs,sdg3_2_mi_abs,sdg3_2_mn_abs,sdg3_3_c_abs,sdg3_3_d_abs,sdg3_3_m_abs,sdg3_3_t_abs,sdg3_3_vih_abs,sdg3_7_fa_abs,sdg4_1_ash_abs,sdg4_1_asm_abs,sdg4_4_es_abs,sdg4_6_alfab_abs,sdg4_c_pci_abs,sdg4_c_pcs_abs,sdg6_1_ca_abs,sdg6_2_cs_abs,sdg7_1_cee_abs,sdg7_1_cep_abs,sdg7_1_elc_abs,sdg7_3_ecep_abs,sdg8_4_mcc_abs,sdg8_5_ph_abs,sdg8_5_pm_abs,sdg8_6_hnn_abs,sdg8_6_mnn_abs,sdg8_10_b_abs,sdg9_1_rutas_abs,sdg9_5_c_abs,sdg9_5_pt_abs,sdg9_c_crdm_abs,sdg9_c_rb_abs,sdg10_2_nhe_abs,sdg11_1_h_abs,sdg11_1_sss_abs,sdg11_2_atc_abs,sdg13_2_ectp_abs,sdg13_2_td_abs,sdg15_1_ap_abs,sdg15_5_pb_abs,sdg16_1_vh_abs,sdg16_6_ce_abs,sdg16_9_nrc_abs,sdg17_5_ip_abs,sdg1_1_ee_norm,sdg1_1_nbi_norm,sdg1_2_pm_norm,sdg1_4_ssb_norm,sdg2_2_dm_norm,sdg2_2_smu_norm,sdg2_4_su_norm,sdg2_4_t_norm,sdg3_1_p_norm,sdg3_2_mi_norm,sdg3_2_mn_norm,sdg3_3_c_norm,sdg3_3_d_norm,sdg3_3_m_norm,sdg3_3_t_norm,sdg3_3_vih_norm,sdg3_7_fa_norm,sdg4_1_ash_norm,sdg4_1_asm_norm,sdg4_4_es_norm,sdg4_6_alfab_norm,sdg4_c_pci_norm,sdg4_c_pcs_norm,sdg5_1_paes_norm,sdg5_1_pej_norm,sdg5_1_ppm_norm,sdg5_5_pp_norm,sdg6_1_ca_norm,sdg6_2_cs_norm,sdg6_3_tar_norm,sdg7_1_cee_norm,sdg7_1_cep_norm,sdg7_1_elc_norm,sdg7_3_ecep_norm,sdg8_4_mcc_norm,sdg8_5_ph_norm,sdg8_5_pm_norm,sdg8_6_hnn_norm,sdg8_6_mnn_norm,sdg8_10_b_norm,sdg8_11_mm_norm,sdg9_1_rutas_norm,sdg9_5_c_norm,sdg9_5_pt_norm,sdg9_c_crdm_norm,sdg9_c_rb_norm,sdg10_2_egini_norm,sdg10_2_ge_norm,sdg10_2_nhe_norm,sdg11_1_h_norm,sdg11_1_sss_norm,sdg11_2_atc_norm,sdg13_1_vcc_norm,sdg13_2_ectp_norm,sdg13_2_td_norm,sdg15_1_ap_norm,sdg15_5_pb_norm,sdg16_1_vh_norm,sdg16_6_ce_norm,sdg16_9_nrc_norm,sdg17_1_il_norm,sdg17_5_ip_norm,indice_ods1,indice_ods2,indice_ods3,indice_ods4,indice_ods5,indice_ods6,indice_ods7,indice_ods8,indice_ods9,indice_ods10,indice_ods11,indice_ods13,indice_ods15,indice_ods16,indice_ods17
count,339.00,339.00,339.00,339.00,339.00,315.00,339.00,339.00,339.00,339.00,339.00,324.00,324.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,338.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,316.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,323.00,339.00,339.00,339.00,315.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,315.00,339.00,339.00,339.00,339.00,339.00,338.00,338.00,337.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,3.390000e+02,339.00,339.00,316.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,323.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,3.390000e+02,339.00,3.390000e+02,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00,339.00

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1c004d15-6318-43c0-9638-a2b00eb26b4d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>